In [1]:
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

consumerTrain = pd.read_csv("train_dataset.csv")
consumerTest = pd.read_csv("test_dataset.csv")

Id = consumerTest["用户编码"]
consumerTest = consumerTest.drop(["用户编码"],axis=1)


# 感觉他妈这个比赛对我们太tm好了，竟然一个缺省值都没有，要哭了
print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(consumerTrain.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(consumerTest.info())

print("-"*50 + "train查看缺失值" + "-"*50)
print(consumerTrain.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(consumerTest.isnull().sum().sort_values(ascending=False).head(10))


--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
用户编码               50000 non-null object
用户实名制是否通过核实        50000 non-null int64
用户年龄               50000 non-null int64
是否大学生客户            50000 non-null int64
是否黑名单客户            50000 non-null int64
是否4G不健康客户          50000 non-null int64
用户网龄（月）            50000 non-null int64
用户最近一次缴费距今时长（月）    50000 non-null int64
缴费用户最近一次缴费金额（元）    50000 non-null float64
用户近6个月平均消费值（元）     50000 non-null float64
用户账单当月总费用（元）       50000 non-null float64
用户当月账户余额（元）        50000 non-null int64
缴费用户当前是否欠费缴费       50000 non-null int64
用户话费敏感度            50000 non-null int64
当月通话交往圈人数          50000 non-null int64
是否经常逛商场的人          50000 non-null int64
近三个月月均商场出现次数       50000 non-null int64
当月是否逛过福州仓山万达       50000 non-null int64
当月是否到过福州山姆会员店      50000 non-null int64
当月是否看电影  

In [2]:
consumerTrain["次数"] = consumerTrain["当月网购类应用使用次数"]+consumerTrain["当月物流快递类应用使用次数"]+consumerTrain["当月金融理财类应用使用总次数"]+consumerTrain["当月视频播放类应用使用次数"]+consumerTrain["当月飞机类应用使用次数"]+consumerTrain["当月火车类应用使用次数"]+consumerTrain["当月旅游资讯类应用使用次数"]
consumerTest["次数"] = consumerTest["当月网购类应用使用次数"]+consumerTest["当月物流快递类应用使用次数"]+consumerTest["当月金融理财类应用使用总次数"]+consumerTest["当月视频播放类应用使用次数"]+consumerTest["当月飞机类应用使用次数"]+consumerTest["当月火车类应用使用次数"]+consumerTest["当月旅游资讯类应用使用次数"]
            
consumerTrain["火车飞机类应用使用次数"] = consumerTrain["当月飞机类应用使用次数"]+consumerTrain["当月火车类应用使用次数"]
consumerTest["火车飞机类应用使用次数"] = consumerTest["当月飞机类应用使用次数"]+consumerTest["当月火车类应用使用次数"]


In [3]:
consumerTrain["是否不良"] = consumerTrain["是否黑名单客户"]+consumerTrain["是否4G不健康客户"]
consumerTest["是否不良"] = consumerTest["是否黑名单客户"]+consumerTest["是否4G不健康客户"]

consumerTrain["是否不良"][consumerTrain["是否不良"]>1] = 1
consumerTest["是否不良"][consumerTest["是否不良"]>1] = 1

In [4]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import model_selection

x = consumerTrain.drop(["信用分","用户编码"],axis=1)
y = consumerTrain.信用分
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=10)


In [35]:
# clf_est =XGBRegressor(random_state=11)
# clf_param_grid = {'n_estimators': [1500],
#                   'learning_rate': [0.1],
#                  'max_depth': [6], 
#                   'min_child_weight': [3,5], 
#                   'gamma':[0.9], 
#                   'subsample':[0.6],
#                   'colsample_bytree':[0.35]}
# clf_grid = model_selection.GridSearchCV(clf_est, clf_param_grid, n_jobs=3, cv=3, verbose=1)
# clf = clf_grid
# clf.fit(x_train, y_train)
# print('BestParams:' + str(clf_grid.best_params_))
print("Training Start")
clf = XGBRegressor(max_depth=5,seed=1314).fit(x_train,y_train)
print("Training Finished")

Training Start
Training Finished


In [36]:

predictions = clf.predict(consumerTest.astype(int))
Submission = pd.DataFrame({'id': Id, 
                           'score': predictions})
Submission["score"] = Submission["score"].astype(int)
Submission.to_csv('SubmissionXGB.csv',index=False,sep=',')
print("I LOVE YOUU")

print("评分：")
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf,x_test,y_test)
print("XGB: ",scores.mean())
from sklearn import metrics

y_pred_xgb = clf.predict(x_test)
# 均方根误差
MSE = metrics.mean_squared_error(y_test,y_pred_xgb)
print(MSE)

I LOVE YOUU
评分：
XGB:  0.7792446218310095
380.314469708212


In [38]:
print(1/(1+MSE))

0.0026225073513869435
